In [1]:
import pickle
import numpy as np
import geopandas as gpd
import numpy as np
import pickle
import boto3

In [2]:
print('Importing B_matrix_weighted_array_0_to_141592.pickle...')
with open(r'intermediate_files/B_matrix_weighted_array_0_to_141592.pickle', 'rb') as handle:
    B_matrix_weighted_array = pickle.load(handle) 

Importing B_matrix_weighted_array_0_to_141592.pickle...


In [2]:
# print('Importing nodes_edges_ucla_big_graph.pickle...')
# with open(r'nodes_edges_ucla_big_graph.pickle', 'rb') as handle:
#     B_matrix_sliced,B_matrix_str_sliced,nodes_coordinates_array = pickle.load(handle)

print('Importing nodes_edges_ucla_access.pickle...')
with open(r'nodes_edges_ucla_access.pickle', 'rb') as handle:
    B_matrix_sliced,B_matrix_str_sliced,nodes_coordinates_array = pickle.load(handle)

Importing nodes_edges_ucla_access.pickle...


In [3]:
# print('Importing ucla_large_centrality_roi.geojson...')
# udf = gpd.read_file('ucla_large_centrality_roi.geojson')

print('Importing uclaaccess_roi_widths_short.geojson...')
udf = gpd.read_file('uclaaccess_roi_widths_short.geojson')

Importing uclaaccess_roi_widths_short.geojson...


In [4]:
len(udf)

267887

In [6]:
B_matrix_sliced.shape

(267889, 7)

In [6]:
count=0

for i in range(B_matrix_weighted_array.shape[0]):
    if np.count_nonzero(B_matrix_weighted_array[i,6:])==0:
        count+=1

percent_updated_B_matrix = (1 - (count / B_matrix_weighted_array.shape[0]))*100

print('Percent of B_matrix updated from script 04_combined_shortest_path_usage.py: ', round(percent_updated_B_matrix, 2), '%')

Percent of B_matrix updated from script 04_combined_shortest_path_usage.py:  93.58 %


In [7]:
oids = np.array(udf['ObjectID']).astype(int)
allocated_B_matrix=np.zeros((len(oids),10))

In [8]:
print('Allocating B_matrix values...')

# Create a dictionary to map oid values to their indices
oid_to_index = {int(oid): idx for idx, oid in enumerate(B_matrix_str_sliced[:,3].astype(int))}

exceptioncounter = 0
for i in range(len(oids)):
    oid = oids[i]
    try:
        idx = oid_to_index[oid]
        allocated_B_matrix[i,:] = B_matrix_weighted_array[idx,6:16]
    except KeyError:
        exceptioncounter += 1

Allocating B_matrix values...


In [9]:
# exceptioncounter=0
# for i in range(len(oids)):
#     try:
#         oid=oids[i]
#         print(oid)
#         idx=np.where(B_matrix_str_sliced[:,3].astype(int) == oid)
#         allocated_B_matrix[i,:]=B_matrix_weighted_array[idx,6:16]
#         print(idx)
#     except:
#         exceptioncounter+=1

In [10]:
print('Updating normalized values in udf_UPDATED_LODES...')

udf_UPDATED_LODES = udf.copy()

udf_UPDATED_LODES["S000_adjusted"]=allocated_B_matrix[:,0]/np.max(allocated_B_matrix[:,0])
udf_UPDATED_LODES["SA01_adjusted"]=allocated_B_matrix[:,1]/np.max(allocated_B_matrix[:,1])
udf_UPDATED_LODES["SA02_adjusted"]=allocated_B_matrix[:,2]/np.max(allocated_B_matrix[:,2])
udf_UPDATED_LODES["SA03_adjusted"]=allocated_B_matrix[:,3]/np.max(allocated_B_matrix[:,3])
udf_UPDATED_LODES["SE01_adjusted"]=allocated_B_matrix[:,4]/np.max(allocated_B_matrix[:,4])
udf_UPDATED_LODES["SE02_adjusted"]=allocated_B_matrix[:,5]/np.max(allocated_B_matrix[:,5])
udf_UPDATED_LODES["SE03_adjusted"]=allocated_B_matrix[:,6]/np.max(allocated_B_matrix[:,6])
udf_UPDATED_LODES["SI01_adjusted"]=allocated_B_matrix[:,7]/np.max(allocated_B_matrix[:,7])
udf_UPDATED_LODES["SI02_adjusted"]=allocated_B_matrix[:,8]/np.max(allocated_B_matrix[:,8])
udf_UPDATED_LODES["SI03_adjusted"]=allocated_B_matrix[:,9]/np.max(allocated_B_matrix[:,9])

Updating normalized values in udf_UPDATED_LODES...


In [11]:
udf_UPDATED_LODES.to_file(r"ucla_large_centrality_roi_UPDATED_LODES_2021.geojson", driver='GeoJSON')

In [25]:
# count how many values in udf_UPDATED_LODES['S000_adjusted'] are 0

S000_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['S000_adjusted']==0])
S000_no_usage_percent = (S000_adjusted_zero_count / len(udf_UPDATED_LODES['S000_adjusted']))*100

SA01_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SA01_adjusted']==0])
SA01_no_usage_percent = (SA01_adjusted_zero_count / len(udf_UPDATED_LODES['SA01_adjusted']))*100

SA02_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SA02_adjusted']==0])
SA02_no_usage_percent = (SA02_adjusted_zero_count / len(udf_UPDATED_LODES['SA02_adjusted']))*100

SA03_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SA03_adjusted']==0])
SA03_no_usage_percent = (SA03_adjusted_zero_count / len(udf_UPDATED_LODES['SA03_adjusted']))*100

SE01_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SE01_adjusted']==0])
SE01_no_usage_percent = (SE01_adjusted_zero_count / len(udf_UPDATED_LODES['SE01_adjusted']))*100

SE02_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SE02_adjusted']==0])
SE02_no_usage_percent = (SE02_adjusted_zero_count / len(udf_UPDATED_LODES['SE02_adjusted']))*100

SE03_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SE03_adjusted']==0])
SE03_no_usage_percent = (SE03_adjusted_zero_count / len(udf_UPDATED_LODES['SE03_adjusted']))*100

SI01_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SI01_adjusted']==0])
SI01_no_usage_percent = (SI01_adjusted_zero_count / len(udf_UPDATED_LODES['SI01_adjusted']))*100

SI02_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SI02_adjusted']==0])
SI02_no_usage_percent = (SI02_adjusted_zero_count / len(udf_UPDATED_LODES['SI02_adjusted']))*100

SI03_adjusted_zero_count = len(udf_UPDATED_LODES[udf_UPDATED_LODES['SI03_adjusted']==0])
SI03_no_usage_percent = (SI03_adjusted_zero_count / len(udf_UPDATED_LODES['SI03_adjusted']))*100

In [27]:
print('Percent of S000_adjusted == 0: ', round(S000_no_usage_percent, 2), '%')
print('Percent of SA01_adjusted == 0: ', round(SA01_no_usage_percent, 2), '%')
print('Percent of SA02_adjusted == 0: ', round(SA02_no_usage_percent, 2), '%')
print('Percent of SA03_adjusted == 0: ', round(SA03_no_usage_percent, 2), '%')
print('Percent of SE01_adjusted == 0: ', round(SE01_no_usage_percent, 2), '%')
print('Percent of SE02_adjusted == 0: ', round(SE02_no_usage_percent, 2), '%')
print('Percent of SE03_adjusted == 0: ', round(SE03_no_usage_percent, 2), '%')
print('Percent of SI01_adjusted == 0: ', round(SI01_no_usage_percent, 2), '%')
print('Percent of SI02_adjusted == 0: ', round(SI02_no_usage_percent, 2), '%')
print('Percent of SI03_adjusted == 0: ', round(SI03_no_usage_percent, 2), '%')

Percent of S000_adjusted == 0:  6.42 %
Percent of SA01_adjusted == 0:  10.64 %
Percent of SA02_adjusted == 0:  7.33 %
Percent of SA03_adjusted == 0:  8.51 %
Percent of SE01_adjusted == 0:  9.91 %
Percent of SE02_adjusted == 0:  8.38 %
Percent of SE03_adjusted == 0:  8.23 %
Percent of SI01_adjusted == 0:  15.92 %
Percent of SI02_adjusted == 0:  12.43 %
Percent of SI03_adjusted == 0:  7.25 %
